<a href="https://colab.research.google.com/github/abhinav3632/googleColab/blob/main/Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
%mkdir Blur_Version_1_Basics_OOPS


In [3]:
 %cd /content/Blur_Version_1_Basics_OOPS

/content/Blur_Version_1_Basics_OOPS


In [4]:
!cmake -B build

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenCV: /usr (found version "4.5.4")
-- Configuring done (0.9s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Blur_Version_1_Basics_OOPS/build


In [5]:
%cd /content/Blur_Version_1_Basics_OOPS/build

/content/Blur_Version_1_Basics_OOPS/build


In [6]:
!make

[ 25%] Building CXX object CMakeFiles/blur_filters.dir/Main.cpp.o
[ 50%] Building CXX object CMakeFiles/blur_filters.dir/Image.cpp.o
[ 75%] Building CXX object CMakeFiles/blur_filters.dir/Functionality.cpp.o
[100%] Linking CXX executable blur_filters
[100%] Built target blur_filters


In [7]:
!./blur_filters

Enter the kernel size (odd number):5
Time Elapsed: 146.73ms


In [12]:
%cd /content/Blur_Version_2_Parallelized

/content/Blur_Version_2_Parallelized


In [13]:
%%writefile CMakeLists.txt
cmake_minimum_required(VERSION 3.5)

# Project Name
project(BlurFilters VERSION 1.0)

# C++ Standard
set(CMAKE_CXX_STANDARD 17)
set(CMAKE_CXX_STANDARD_REQUIRED True)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -O3")

# Find OpenCV Package
find_package(OpenCV REQUIRED)
include_directories(${OpenCV_INCLUDE_DIRS})
include_directories(${PROJECT_SOURCE_DIR}/Exceptions)

# Add the Executable
add_executable(blur_filters Main.cpp Image.cpp Functionality.cpp)

# Link OpenCV libraries
target_link_libraries(blur_filters ${OpenCV_LIBS} pthread)


Writing CMakeLists.txt


In [15]:
%%writefile FilterType.h
#ifndef FILTERTYPE_H
#define FILTERTYPE_H

/*
    Enum Class for the type of filter
*/

enum class FilterType{
    Box,
    Blur,
    Gaussian,
    Median,
    Custom
};

#endif // FILTERTYPE_H


Overwriting FilterType.h


In [16]:
%%writefile Functionality.cpp
#include "Functionality.h"

/*
    applyFilter() takes a constant Image object, int variable and a enum class type variable
    by l-value reference and performs the said blurring operations on it.
*/
void applyFilter(const Image &img, int kernelSize, FilterType type)
{
    /* Check if the kernelSize entered is odd or not */
    if(kernelSize % 2 == 0 || kernelSize < 1){
        throw InvalidKernelSizeError("Error: Kernel size must be a positive odd number.", std::future_errc::no_state);
    }

    /* Resultant Matrix */
    cv::Mat result;
    std::string windowName;
    //Define a custom kernel (eg: a simple averaging kernel)
    cv::Mat customKernel = cv::Mat::ones(kernelSize, kernelSize, CV_32F)/ (float)(kernelSize*kernelSize);

    switch(type){
        case FilterType::Box:
            cv::boxFilter(img.src_rgb(), result, -1, cv::Size(kernelSize, kernelSize));
            windowName = "Box Filter";
            break;

        case FilterType::Blur:
            cv::blur(img.src_rgb(), result, cv::Size(kernelSize, kernelSize));
            windowName = "Blur Filter";
            break;

        case FilterType::Gaussian:
            cv::GaussianBlur(img.src_rgb(), result, cv::Size(kernelSize, kernelSize), 0);
            windowName = "Gaussian Filter";
            break;

        case FilterType::Median:
            cv::medianBlur(img.src_rgb(), result, kernelSize);
            windowName = "Median Filter";
            break;

        case FilterType::Custom:
            cv::filter2D(img.src_rgb(), result, -1, customKernel);
            windowName = "Custom Filter";
            break;

        default:
            throw std::invalid_argument("Error: Invalid Filter Type.");
    }
    /* Check whether a valid windowName was used*/
    if(windowName.size() == 0 || windowName == " " || windowName == ""){
        throw EmptyWindowNameError("Error: Window Name cannot be empty" , std::future_errc::no_state);
    }
    /* Display function is executed to display the result*/
    img.display(result, windowName);
}


Writing Functionality.cpp


In [17]:
%%writefile Functionality.h
#ifndef FUNCTIONALITY_H
#define FUNCTIONALITY_H

#include "Image.h"
#include "Exceptions/InvalidKernelSizeError.h"
#include "Exceptions/EmptyWindowNameError.h"
#include "FilterType.h"

/*
    A function to apply the filter on the
    given image
    Input: given rgb image, kernel size
    Output: blurred output image
*/

void applyFilter(const Image &img, int kernelSize, FilterType type);


#endif // FUNCTIONALITY_H


Writing Functionality.h


In [18]:
%%writefile Image.h
#ifndef IMAGE_H
#define IMAGE_H

#include <iostream>
#include <memory>
#include <opencv2/opencv.hpp>
#include "Exceptions/EmptyImagePathError.h"
#include "Exceptions/EmptyImageError.h"

class Image
{
private:
    cv::Mat m_src_rgb; // Source image in rgb format
    static std::unique_ptr<Image> instance; // Singleton instance

    //Private Constructor
    Image() = default; // Default constructor
    Image(const std::string &imagePath); // Parameterized Constructor to load image

public:
    Image(const Image&) = delete; // Deleted copy constructor
    Image& operator=(const Image&) = delete; // Deleted copy assignment
    Image(Image&&) = delete; // Deleted move constructor
    Image& operator=(Image&&) = delete; // Deleted move assignment
    ~Image() = default;

    static Image& getInstance(const std::string &imagePath = " ");

    cv::Mat src_rgb() const { return m_src_rgb;}

    void display(const cv::Mat &image, const std::string &windowName) const; // Display image
    static void closeAllWindows(); // Close all openCv windows
};

#endif // IMAGE_H


Writing Image.h


In [22]:
%%writefile Image.cpp
#include "Image.h"

std::unique_ptr<Image> Image::instance = nullptr;

/*
    Image() constructor which takes std::string variable by l-value reference and initializes
    the Image object with the image in the given path
*/
Image::Image(const std::string &imagePath)
{
    /* Check if a valid image Path was provided */
    if(imagePath.size() == 0 || imagePath == " " || imagePath == ""){
        throw EmptyImagePathError("Error: Invalid Image Path", std::future_errc::no_state);
    }
    m_src_rgb = cv::imread(imagePath, cv::IMREAD_COLOR);
    /* Check if the image is valid */
    if(m_src_rgb.empty()){
        throw EmptyImageError("Error: Unable to load image from given path.", std::future_errc::no_state);
        exit(1);
    }
}

/*
    getInstance() is a function which takes one constant std::string data type variable by l-value reference and the function returns a Image (class) by reference.
*/
Image& Image::getInstance(const std::string &imagePath){
    if(!instance){
        instance.reset(new Image(imagePath));
    }
    return *instance;
}

/*  display() is a function which takes a Matrix object and a std::string data type variables by l-value
    reference and displays the resultant using cv::imshow */
void Image::display(const cv::Mat &image, const std::string &windowName) const{
    // cv::imshow(windowName, image);
    std::string fileName = "/content/Blur_Version_2_Parallelized/saved/" + windowName + ".png";
    cv::imwrite(fileName, image);
}

/*  closeAllWindows() is a function which takes no arguments, and is for destroying the windows after
    execution */
void Image::closeAllWindows()
{
    cv::waitKey(0);
    cv::destroyAllWindows();
}


Overwriting Image.cpp


In [20]:
%%writefile Main.cpp
#include "Functionality.h"
#include "Time.h"
#include <future>
#include <vector>

int main()
{
    try
    {
        // Get the Singleton instance
        Image& img = Image::getInstance("/content/Blur_Version_2_Parallelized/lena.jpg");

        img.display(img.src_rgb(), "Original Image");

        // Taking input of kernelSize
        int kernelSize;
        std::cout << "Enter the kernel size (odd number):";
        std::cin >> kernelSize;

        std::vector<std::future<void>> futures;
        {
            // timer object for checking the implementation time elapsed
            Time timer;
            try
            {
                futures.push_back(std::async(std::launch::async, applyFilter, std::ref(img), kernelSize, FilterType::Box));
                futures.push_back(std::async(std::launch::async, applyFilter, std::ref(img), kernelSize, FilterType::Blur));
                futures.push_back(std::async(std::launch::async, applyFilter, std::ref(img), kernelSize, FilterType::Gaussian));
                futures.push_back(std::async(std::launch::async, applyFilter, std::ref(img), kernelSize, FilterType::Median));
                futures.push_back(std::async(std::launch::async, applyFilter, std::ref(img), kernelSize, FilterType::Custom));

                for (auto &f : futures)
                {
                    f.get(); // Wait for each async task to complete
                }
            }
            catch (const InvalidKernelSizeError &e)
            {
                std::cerr << e.what() << '\n';
            }
            catch (const EmptyWindowNameError &e)
            {
                std::cerr << e.what() << '\n';
            }
        }
        img.closeAllWindows();
    }
    catch (const EmptyImagePathError &e)
    {
        std::cerr << e.what() << '\n';
    }
    catch (const EmptyImageError &e)
    {
        std::cerr << e.what() << '\n';
    }
    return EXIT_SUCCESS;
}

Writing Main.cpp


In [27]:
%%writefile Time.h
#ifndef TIME_H
#define TIME_H

#include <iostream>
#include <memory>

#include <chrono>

class Time{
    private:
        std::chrono::time_point<std::chrono::high_resolution_clock> m_StartTimePoint;

        void Stop(){
            auto endTimePoint = std::chrono::high_resolution_clock::now();

            auto start = std::chrono::time_point_cast<std::chrono::microseconds>(m_StartTimePoint).time_since_epoch().count();
            auto end = std::chrono::time_point_cast<std::chrono::microseconds>(endTimePoint).time_since_epoch().count();

            auto duration = end - start;
            double ms = duration * 0.001;

            std::cout << "Time Elapsed: " << ms << "ms\n";
        }

    public:
    Time(){
        m_StartTimePoint = std::chrono::high_resolution_clock::now();
    }

    ~Time(){
        Stop();
    }
};

#endif // TIME_H


Writing Time.h


In [29]:
!cmake -B build

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenCV: /usr (found version "4.5.4")
-- Configuring done (0.4s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Blur_Version_2_Parallelized/build


In [30]:
%cd build

/content/Blur_Version_2_Parallelized/build


In [31]:
!make

[ 25%] Building CXX object CMakeFiles/blur_filters.dir/Main.cpp.o
[ 50%] Building CXX object CMakeFiles/blur_filters.dir/Image.cpp.o
[ 75%] Building CXX object CMakeFiles/blur_filters.dir/Functionality.cpp.o
[100%] Linking CXX executable blur_filters
[100%] Built target blur_filters


In [34]:
!./blur_filters

Enter the kernel size (odd number):5
Time Elapsed: 81.268ms
